In [1]:
###IMPORT MODULES
import sys
import os
sys.path.append(os.path.abspath('../..'))

import numpy as np 
from tools.component_tools import Component
from tools.component_tools import Fluid
from tools.component_tools import Membrane
import tools.materials as materials
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from tools.component_tools import BreedingBlanket
import mph 
from tools.component_tools import Geometry


U0=2
eff_coms_v=np.array([])
eff_py=np.array([])
client = mph.start()
model = client.load('PAV_onlysolid.mph')  


# U0=1.5
T=800
c0=1E-2
mat=materials.Flibe(T)
T_hot_prim=900
T_cold_prim=800
Q=1E9
d_hyd=1E-2
L=5
D=1E-9
thick=2E-3
K_S=20
k_d=1E6
k_r=1E6
K_H=1E-2
flibe=Fluid(T=T, Solubility=K_H, MS=False,D=mat.D, d_Hyd=d_hyd ,mu=mat.mu,rho=mat.rho,U0=U0,k=mat.k,
        cp=mat.cp)
var_vec=[T,c0*K_S/K_H,d_hyd,L,D,thick,K_S,K_H,mat.D,U0]
var_vec_str=['T','c0','d_hyd','L','D_St','thick','K_S','K_H','D_flibe','U0']
Steel = Membrane( T=T,
        D=D,
        thick=thick,
        K_S=K_S,
        k_d=k_d,
        k_r=k_r,k=21)
geom_pav=Geometry(L=L, D=d_hyd,thick=thick )
PAV = Component(c_in=c0,  fluid=flibe, membrane=Steel,geometry=geom_pav)
PAV.get_solid_inventory()
for i,var in enumerate(var_vec):
        model.parameter(var_vec_str[i], str(var))
model.mesh()
a=model.parameters()
# model.save()

model.solve()
# Evaluate the results
eff_coms = model.evaluate(['r', 'z', 'inv'], 'Study 1//Solution 1')

# Remove NaN values from the 'inv' component
inv_values = eff_coms[-1]  # Assuming 'inv' is the last component
inv_values = inv_values[~np.isnan(inv_values)]

# Calculate the average of the cleaned 'inv' values
eff_coms_avg = np.average(inv_values)

# Append the average to the eff_coms_v array
eff_coms_v = np.append(eff_coms_v, eff_coms_avg)

# Print the results
print("PAV", (PAV.membrane.inv), "Coms=", eff_coms_avg, "%", "diff=", PAV.membrane.inv - eff_coms_avg, "%", "rel_diff=", (abs(PAV.membrane.inv - eff_coms_avg) / PAV.membrane.inv) * 100, "%")

# Append the PAV membrane inventory to the eff_py array


PAV 0.003270964016294212 Coms= 0.0033502449679335146 % diff= -7.928095163930267e-05 % rel_diff= 2.4237793887174215 %


In [2]:

print("ratio", PAV.membrane.inv/eff_coms_avg)

# Remove the model from the client
client.remove(model)


ratio 0.9763357747274808
